<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparation" data-toc-modified-id="Preparation-1">Preparation</a></span><ul class="toc-item"><li><span><a href="#Load-files" data-toc-modified-id="Load-files-1.1">Load files</a></span></li></ul></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span><ul class="toc-item"><li><span><a href="#Helper-functions" data-toc-modified-id="Helper-functions-2.1">Helper functions</a></span></li><li><span><a href="#Metric-functions" data-toc-modified-id="Metric-functions-2.2">Metric functions</a></span><ul class="toc-item"><li><span><a href="#All-metrics" data-toc-modified-id="All-metrics-2.2.1">All metrics</a></span></li><li><span><a href="#Gaze-data" data-toc-modified-id="Gaze-data-2.2.2">Gaze data</a></span></li><li><span><a href="#Fixation-data" data-toc-modified-id="Fixation-data-2.2.3">Fixation data</a></span></li><li><span><a href="#Event-data" data-toc-modified-id="Event-data-2.2.4">Event data</a></span></li></ul></li></ul></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-3">Analysis</a></span><ul class="toc-item"><li><span><a href="#Baseline-Analysis" data-toc-modified-id="Baseline-Analysis-3.1">Baseline Analysis</a></span></li><li><span><a href="#FXD-Analysis" data-toc-modified-id="FXD-Analysis-3.2">FXD Analysis</a></span></li><li><span><a href="#EVD-Analysis" data-toc-modified-id="EVD-Analysis-3.3">EVD Analysis</a></span></li><li><span><a href="#GZD-Analysis" data-toc-modified-id="GZD-Analysis-3.4">GZD Analysis</a></span></li></ul></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-4">Analysis</a></span><ul class="toc-item"><li><span><a href="#Additional-data" data-toc-modified-id="Additional-data-4.1">Additional data</a></span><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-4.1.1">Overview</a></span></li><li><span><a href="#Top-10" data-toc-modified-id="Top-10-4.1.2">Top 10</a></span></li><li><span><a href="#Bottom-10" data-toc-modified-id="Bottom-10-4.1.3">Bottom 10</a></span></li></ul></li></ul></li></ul></div>

<h1>Eye Gaze Data Processing<span class="tocSkip"></span></h1>

Following metrics from: https://github.com/TheD2Lab/Eye.Tracking.Data.Analysis.For.Tobii.2150/tree/master/src/analysis

- Isaac Cortes
- Reynaldo Suarez
- Chris
- Janki

In [1]:
import pandas as pd
import numpy as np
import random 
import math

# Preparation

## Load files

In [2]:
def load_files(participant = 1):
    
    route = f'datasets/p{participant}/p{participant}'
    
    # Load one of the files to create functions
    baseline_cols = ['number', 'time', 'l_screen_x', 'l_screen_y', 'l_cam_x', 'l_cam_y', 'l_distance', 
                'l_pupil', 'l_code', 'r_screen_x', 'r_screen_y', 'r_cam_x', 'r_cam_y', 'r_distance', 
                'r_pupil', 'r_code']

    baseline = pd.read_csv(f'{route}GZD.txt', sep='\t', names = baseline_cols)
    
    # Load FXD graph and tree
    fxd_cols = ['number', 'time', 'duration', 'screen_x', 'screen_y']

    fxd_graph = pd.read_csv(f'{route}.graphFXD.txt', sep='\t', names = fxd_cols)
    fxd_tree = pd.read_csv(f'{route}.treeFXD.txt', sep='\t', names = fxd_cols)

    # Load EVD graph and tree
    evd_cols = ['time', 'event', 'event_key', 'data1', 'data2', 'description']

    evd_graph = pd.read_csv(f'{route}.graphEVD.txt', sep='\t', names = evd_cols)
    evd_tree = pd.read_csv(f'{route}.treeEVD.txt', sep='\t', names = evd_cols)
    
    # Load GZD graph and tree
    gzd_cols = ['number', 'time', 'l_screen_x', 'l_screen_y', 'l_cam_x', 'l_cam_y', 'l_distance', 
                'l_pupil', 'l_code', 'r_screen_x', 'r_screen_y', 'r_cam_x', 'r_cam_y', 'r_distance', 
                'r_pupil', 'r_code']

    gzd_graph = pd.read_csv(f'{route}.graphGZD.txt', sep='\t', names = gzd_cols)
    gzd_tree = pd.read_csv(f'{route}.treeGZD.txt', sep='\t', names = gzd_cols)

    return baseline, fxd_graph, fxd_tree, evd_graph, evd_tree, gzd_graph, gzd_tree


In [3]:
# Not all numbers 1-36 are participants in folder, use random choose instead with the folder contents
participant = random.randint(1,36)
print(f'Participant: {participant}')

baseline, fxd_graph, fxd_tree, evd_graph, evd_tree, gzd_graph, gzd_tree = load_files(participant)

Participant: 20


# Functions

## Helper functions

$$d = \sqrt{(x_2 - x_1)^2 + (y_2-y_1)^2}$$

In [4]:
def distance(row):
    x1, y1 = row['x'], row['y']
    x2, y2 =  row['next_x'], row['next_y']
    dist = math.sqrt(math.pow((x2-x1), 2) + math.pow((y2-y1), 2))
    row['dist'] = dist
    
    return row

$$dur = T2-(T1+D1)$$

In [5]:
def duration(row):
    
    t1, d1 = row['time'], row['duration']
    t2 = row['next_time']
    duration = t2 - (t1+d1)
    row['duration_between_fixations'] = duration
    
    return row 

## Metric functions

### All metrics

In [6]:
def get_metrics_df(row):
    
    participant = ''.join(filter(str.isdigit, row['ID']))
    print(participant)
    
    visualization = row['Visualization']
    
    # Load data for participant
    baseline, fxd_graph, fxd_tree, evd_graph, evd_tree, gzd_graph, gzd_tree = load_files(participant)
    
    # Baseline gaze data
    base_avg_size_left, base_avg_size_right, base_avg_size_both = get_gaze_metrics(baseline)
    # Graph data
    if visualization == 1:
        # Fixations data
        total_fixations, sum_fixation_duration, mean_fixation_duration, std_fixation_duration = get_fixation_metrics(fxd_graph)
        # Saccade length data
        total_saccades, sum_saccade_length, mean_saccade_length, std_saccade_length = get_saccade_length_metrics(fxd_graph)
        # Saccade duration data
        sum_saccade_duration, mean_saccade_duration, std_saccade_duration = get_saccade_durations_metrics(fxd_graph)
        # Event data
        total_left_clicks, mean_time_between_clicks, std_time_between_clicks = get_event_metrics(evd_graph)
        # Gaze data
        avg_size_left, avg_size_right, avg_size_both = get_gaze_metrics(gzd_graph)
    # Tree data
    else:
        # Fixations data
        total_fixations, sum_fixation_duration, mean_fixation_duration, std_fixation_duration = get_fixation_metrics(fxd_tree)
        # Saccade length data
        total_saccades, sum_saccade_length, mean_saccade_length, std_saccade_length = get_saccade_length_metrics(fxd_tree)
        # Saccade duration data
        sum_saccade_duration, mean_saccade_duration, std_saccade_duration = get_saccade_durations_metrics(fxd_tree)
        # Event data
        total_left_clicks, mean_time_between_clicks, std_time_between_clicks = get_event_metrics(evd_tree)
        # Gaze data
        avg_size_left, avg_size_right, avg_size_both = get_gaze_metrics(gzd_tree)
    
    cols = ['base_avg_size_left', 'base_avg_size_right', 
            'base_avg_size_both', 'total_fixations', 'sum_fixation_duration', 
            'mean_fixation_duration', 'std_fixation_duration', 'total_saccades', 
            'sum_saccade_length', 'mean_saccade_length', 'std_saccade_length', 
            'sum_saccade_duration', 'mean_saccade_duration', 'std_saccade_duration', 
            'total_left_clicks', 'mean_time_between_clicks', 'std_time_between_clicks', 
            'avg_size_left', 'avg_size_right', 'avg_size_both']
    
    data = [base_avg_size_left, base_avg_size_right, 
            base_avg_size_both, total_fixations, sum_fixation_duration, 
            mean_fixation_duration, std_fixation_duration, total_saccades, 
            sum_saccade_length, mean_saccade_length, std_saccade_length, sum_saccade_duration, 
            mean_saccade_duration, std_saccade_duration, total_left_clicks, 
            mean_time_between_clicks, std_time_between_clicks, 
            avg_size_left, avg_size_right, avg_size_both]
    
    for i in range(len(cols)):
        row[cols[i]] = data[i]
    
    return row

### Gaze data

In [7]:
def get_gaze_metrics(df):
    
    # A code with 0 indicates the eye tracker was confdident with this data
    # Filtering only records where both pupil sizes are valid
    df = df[(df['l_code'] == 0) & (df['r_code'] == 0)]
    avg_size_left = df["l_pupil"].mean()
    avg_size_right = df["r_pupil"].mean()
    avg_size_both = pd.concat([df["r_pupil"],df["l_pupil"]]).mean()
    
    return avg_size_left, avg_size_right, avg_size_both

In [8]:
def print_gaze_metrics(df):
    avg_size_left, avg_size_right, avg_size_both = get_gaze_metrics(df)

    print(f'Avg. pupil size of the left eye: {avg_size_left:.4f}')
    print(f'Avg. pupil size of the right eye: {avg_size_right:.4f}')
    print(f'Avg. pupil size of both eyes: {avg_size_both:.4f}')

### Fixation data

In [9]:
def get_fixation_metrics(df):
    
    total_fixations = len(df)
    sum_fixation_duration_sec = df["duration"].sum() / 1000
    mean_fixation_duration = df["duration"].mean()
    std_fixation_duration = df["duration"].std()
    
    return total_fixations, sum_fixation_duration_sec, mean_fixation_duration, std_fixation_duration
    
def get_saccade_length_metrics(df):
    
    # Get distances of all points
    coords = df[['screen_x','screen_y']].copy()
    coords = coords.rename({'screen_x':'x','screen_y':'y'}, axis='columns')
    coords['next_x'] = coords['x'].shift(-1)
    coords['next_y'] = coords['y'].shift(-1)
    coords = coords.apply(distance, axis=1)
    coords.dropna(inplace=True)
    
    total_saccades = len(coords)
    sum_saccade_length = coords['dist'].sum()
    mean_saccade_length = coords['dist'].mean()
    std_saccade_length = coords['dist'].std()
        
    return total_saccades, sum_saccade_length, mean_saccade_length, std_saccade_length

def get_saccade_durations_metrics(df):
    
    # Get duration of all saccades
    saccadeDetails = df[['time','duration']].copy()
    saccadeDetails['next_time'] = saccadeDetails['time'].shift(-1)
    saccadeDetails['next_duration'] = saccadeDetails['duration'].shift(-1)
    saccadeDetails = saccadeDetails.apply(duration, axis=1)
    saccadeDetails.dropna(inplace=True)
    
    sum_saccade_duration_sec = saccadeDetails['duration_between_fixations'].sum() / 1000
    mean_saccade_duration = saccadeDetails['duration_between_fixations'].mean()
    std_saccade_duration = saccadeDetails['duration_between_fixations'].std()
    
    return sum_saccade_duration_sec, mean_saccade_duration, std_saccade_duration

In [10]:
def print_fixation_metrics(df):
    # Fixations
    total_fixations, sum_fixation_duration_sec, mean_fixation_duration, std_fixation_duration = get_fixation_metrics(df)
    print(f'{"="*10} {"Fixations":^20} {"="*10}')
    print(f'Total number of fixations: {total_fixations}')
    print(f'Sum of all fixation durations: {sum_fixation_duration_sec}s')
    print(f'Sum of all fixation durations: {sum_fixation_duration_sec / 60 :.2f}min')
    print(f'Mean fixation duration: {mean_fixation_duration:.2f}ms')
    print(f'StDev fixation duration: {std_fixation_duration:.2f}ms')

    # Saccade 
    total_saccades, sum_saccade_length, mean_saccade_length, std_saccade_length = get_saccade_length_metrics(df)
    print(f'\n{"="*10} {"Saccade lengths":^20} {"="*10}')
    print(f'Total number of saccades: {total_saccades}')
    print(f'Sum of all saccade lengths: {sum_saccade_length:.2f}px')
    print(f'Mean saccade length: {mean_saccade_length:.2f}px')
    print(f'StDev saccade length: {std_saccade_length:.2f}px')

    # Durations
    sum_saccade_duration_sec, mean_saccade_duration, std_saccade_duration = get_saccade_durations_metrics(df)
    print(f'\n{"="*10} {"Saccade durations":^20} {"="*10}')
    print(f'Sum of all saccade durations: {sum_saccade_duration_sec:.2f}s')
    print(f'Sum of all saccade durations: {sum_saccade_duration_sec / 60 :.2f}min')
    print(f'Mean saccade duration: {mean_saccade_duration:.2f}ms')
    print(f'StDev saccade duration: {std_saccade_duration:.2f}ms')

### Event data

In [11]:
def get_event_metrics(df):
    
    lclicks = evd_graph[evd_graph['event'] == 'LMouseButton']
    lclicks = lclicks[['time','data1','data2']]
    lclicks['next_time'] = lclicks['time'].shift(-1)
    lclicks['time_between'] = lclicks['next_time'] - lclicks['time']
    lclicks.dropna(inplace=True)

    total_L_clicks = len(lclicks)
    avg_time_between_clicks_sec = lclicks['time_between'].mean() / 1000
    std_time_between_clicks_sec = lclicks['time_between'].std() / 1000
    
    return total_L_clicks, avg_time_between_clicks_sec, std_time_between_clicks_sec

In [12]:
def print_event_metrics(df):
    
    total_L_clicks, avg_time_between_clicks_sec, std_time_between_clicks_sec = get_event_metrics(df)
    
    print(f'Total number of L mouse clicks: {total_L_clicks}')
    print(f'Avg. amount of time between clicks: {avg_time_between_clicks_sec:.2f}s')
    print(f'StDev amount of time between clicks: {std_time_between_clicks_sec:.2f}s')

# Analysis

## Baseline Analysis

- average pupil size of left eye; 
- average pupil size of right eye; 
- average pupil size of both eyes.

In [13]:
baseline.head()

,number,time,l_screen_x,l_screen_y,l_cam_x,l_cam_y,l_distance,l_pupil,l_code,r_screen_x,r_screen_y,r_cam_x,r_cam_y,r_distance,r_pupil,r_code
0,1,1,-1280,-1024,-1.0,-1.0,-1.0,-1.0,4,-1280,-1024,-1.0,-1.0,-1.0,-1.0,4
1,21,2,-1280,-1024,-1.0,-1.0,-1.0,-1.0,4,-1280,-1024,-1.0,-1.0,-1.0,-1.0,4
2,41,3,-1280,-1024,-1.0,-1.0,-1.0,-1.0,4,-1280,-1024,-1.0,-1.0,-1.0,-1.0,4
3,61,4,-1280,-1024,-1.0,-1.0,-1.0,-1.0,4,-1280,-1024,-1.0,-1.0,-1.0,-1.0,4
4,81,5,-1280,-1024,-1.0,-1.0,-1.0,-1.0,4,-1280,-1024,-1.0,-1.0,-1.0,-1.0,4


In [14]:
%%time
print_gaze_metrics(baseline)

Avg. pupil size of the left eye: 3.4281
Avg. pupil size of the right eye: 3.2167
Avg. pupil size of both eyes: 3.3224
Wall time: 19.1 ms


## FXD Analysis

**Fixations**
- total number of fixations
- sum of all fixation duration
- mean duration
- StDev of durations

**Saccade lengths**
- total number of saccades
- sum of all saccade length
- mean saccade length
- StDev of saccade lengths

**Saccade durations**
- sum of all saccade durations
- mean saccade duration
- StDev of saccade durations

**Research**
- scanpath duration
- fixation to saccade ratio

In [15]:
fxd_graph.head()

,number,time,duration,screen_x,screen_y
0,1,328,239,606,401
1,2,787,160,359,215
2,3,966,439,277,205
3,4,1424,200,501,190
4,5,1644,159,466,185


In [16]:
%%time
print_fixation_metrics(fxd_graph)

==========      Fixations       ==========
Total number of fixations: 4349
Sum of all fixation durations: 1068.897s
Sum of all fixation durations: 17.81min
Mean fixation duration: 245.78ms
StDev fixation duration: 239.06ms

==========   Saccade lengths    ==========
Total number of saccades: 4348
Sum of all saccade lengths: 486544.58px
Mean saccade length: 111.90px
StDev saccade length: 151.05px

==========  Saccade durations   ==========
Sum of all saccade durations: 786.85s
Sum of all saccade durations: 13.11min
Mean saccade duration: 180.97ms
StDev saccade duration: 470.00ms
Wall time: 5.5 s


In [17]:
fxd_tree.head()

,number,time,duration,screen_x,screen_y
0,1,102,140,749,305
1,2,421,140,684,432
2,3,1219,120,688,279
3,4,1498,160,546,216
4,5,1678,179,515,195


In [18]:
%%time
print_fixation_metrics(fxd_tree)

==========      Fixations       ==========
Total number of fixations: 468
Sum of all fixation durations: 99.006s
Sum of all fixation durations: 1.65min
Mean fixation duration: 211.55ms
StDev fixation duration: 144.21ms

==========   Saccade lengths    ==========
Total number of saccades: 467
Sum of all saccade lengths: 45009.89px
Mean saccade length: 96.38px
StDev saccade length: 139.63px

==========  Saccade durations   ==========
Sum of all saccade durations: 481.20s
Sum of all saccade durations: 8.02min
Mean saccade duration: 1030.40ms
StDev saccade duration: 11787.22ms
Wall time: 577 ms


## EVD Analysis

- total number of L mouse clicks. 
- avg time between clicks.
- std time between clicks.

In [19]:
evd_graph.head()

,time,event,event_key,data1,data2,description
0,21912,LMouseButton,1,1112,752,NaN
1,26118,LMouseButton,1,1081,631,NaN
2,29028,LMouseButton,1,1117,664,NaN
3,30337,LMouseButton,1,1070,628,NaN
4,34431,LMouseButton,1,1068,626,NaN


In [20]:
%%time
print_event_metrics(evd_graph)

Total number of L mouse clicks: 482
Avg. amount of time between clicks: 3.78s
StDev amount of time between clicks: 6.53s
Wall time: 7 ms


In [21]:
evd_tree.head()

,time,event,event_key,data1,data2,description
0,12164,LMouseButton,1,41,578,NaN
1,20399,LMouseButton,1,678,575,NaN
2,22822,LMouseButton,1,696,592,NaN
3,27048,LMouseButton,1,61,593,NaN
4,32680,LMouseButton,1,572,213,NaN


In [22]:
%%time
print_event_metrics(evd_tree)

Total number of L mouse clicks: 482
Avg. amount of time between clicks: 3.78s
StDev amount of time between clicks: 6.53s
Wall time: 6 ms


## GZD Analysis

- average pupil size of left eye;
- average pupil size of right eye;
- average pupil size of both eyes.

In [23]:
gzd_graph.head()

,number,time,l_screen_x,l_screen_y,l_cam_x,l_cam_y,l_distance,l_pupil,l_code,r_screen_x,r_screen_y,r_cam_x,r_cam_y,r_distance,r_pupil,r_code
0,9,1,-1280,-1024,0.713,0.512,-1.000,-1.000,4,-1280,-1024,0.489,0.521,-1.000,-1.000,4
1,29,2,-1280,-1024,-1.000,-1.000,-1.000,-1.000,4,-1280,-1024,-1.000,-1.000,-1.000,-1.000,4
2,49,3,732,432,0.714,0.512,754.336,3.402,0,738,389,0.490,0.520,751.802,3.240,0
3,69,4,764,449,0.714,0.512,754.336,3.518,0,750,426,0.490,0.520,751.802,3.227,0
4,89,5,742,434,0.714,0.512,754.336,3.386,0,752,369,0.490,0.520,751.802,3.125,0


In [24]:
%%time
print_gaze_metrics(gzd_graph)

Avg. pupil size of the left eye: 3.2417
Avg. pupil size of the right eye: 3.0284
Avg. pupil size of both eyes: 3.1351
Wall time: 17 ms


In [25]:
gzd_tree.head()

,number,time,l_screen_x,l_screen_y,l_cam_x,l_cam_y,l_distance,l_pupil,l_code,r_screen_x,r_screen_y,r_cam_x,r_cam_y,r_distance,r_pupil,r_code
0,3,1,-1280,-1024,-1.000,-1.000,-1.0,-1.0,4,-1280,-1024,-1.000,-1.000,-1.0,-1.0,4
1,23,2,-1280,-1024,0.766,0.516,-1.0,-1.0,4,-1280,-1024,0.539,0.526,-1.0,-1.0,4
2,43,3,-1280,-1024,-1.000,-1.000,-1.0,-1.0,4,-1280,-1024,-1.000,-1.000,-1.0,-1.0,4
3,63,4,-1280,-1024,0.766,0.516,-1.0,-1.0,4,-1280,-1024,0.539,0.526,-1.0,-1.0,4
4,83,5,-1280,-1024,0.766,0.517,-1.0,-1.0,4,-1280,-1024,0.539,0.526,-1.0,-1.0,4


In [26]:
%%time
print_gaze_metrics(gzd_tree)

Avg. pupil size of the left eye: 3.3291
Avg. pupil size of the right eye: 3.1632
Avg. pupil size of both eyes: 3.2462
Wall time: 7.03 ms


# Analysis

## Additional data

In [27]:
add_data = pd.read_csv('datasets/additional_participant_data.csv')

In [28]:
add_data.head()

,ID,Ontologies,Visualization,Task_Success,Time_On_Task
0,p1,1,1,0.608696,26
1,p1,2,2,0.521739,38
2,p3,1,1,0.478261,10
3,p3,2,2,0.391304,50
4,p5,1,1,0.521739,22


### Overview

In [29]:
add_data.groupby(by=['Ontologies','Visualization']).mean()

Task_Success  Time_On_Task
Ontologies Visualization                            
1          1                  0.494565     17.625000
           2                  0.501449     22.000000
2          1                  0.513043     24.666667
           2                  0.407609     35.187500

### Top 10

In [30]:
top10 = add_data.copy()
top10.sort_values(by=['Task_Success','Time_On_Task'], ascending=[False,True], inplace=True)
top10.reset_index(drop=True, inplace=True)
top10 = top10.head(10)
top10

,ID,Ontologies,Visualization,Task_Success,Time_On_Task
0,p6,1,2,0.869565,26
1,p18,1,2,0.739130,21
2,p24,2,1,0.739130,25
3,p6,2,1,0.739130,43
4,p12,2,1,0.695652,11
5,p35,1,1,0.608696,9
6,p10,1,2,0.608696,17
7,p17,2,2,0.608696,22
8,p1,1,1,0.608696,26
9,p34,1,2,0.608696,38


In [31]:
top10.groupby(by=['Ontologies','Visualization']).mean()

Task_Success  Time_On_Task
Ontologies Visualization                            
1          1                  0.608696     17.500000
           2                  0.706522     25.500000
2          1                  0.724638     26.333333
           2                  0.608696     22.000000

In [32]:
%%time
top10.apply(get_metrics_df, axis=1)

6
18
24
6
12
35
10
17
1
34
Wall time: 53.4 s


,ID,Ontologies,Visualization,Task_Success,Time_On_Task,base_avg_size_left,base_avg_size_right,base_avg_size_both,total_fixations,sum_fixation_duration,...,std_saccade_length,sum_saccade_duration,mean_saccade_duration,std_saccade_duration,total_left_clicks,mean_time_between_clicks,std_time_between_clicks,avg_size_left,avg_size_right,avg_size_both
0,p6,1,2,0.869565,26,3.298531,3.454790,3.376660,6752,1355.507,...,175.946321,1263.480,187.154496,258.299289,482,3.783077,6.528288,3.607497,3.827016,3.717256
1,p18,1,2,0.739130,21,2.469319,2.319605,2.394462,3559,651.412,...,204.923290,964.344,271.035413,402.456275,482,3.783077,6.528288,2.801013,2.642901,2.721957
2,p24,2,1,0.739130,25,3.604752,3.580405,3.592578,2522,352.379,...,220.248797,1610.727,638.923840,897.527182,482,3.783077,6.528288,3.864160,3.737860,3.801010
3,p6,2,1,0.739130,43,3.298531,3.454790,3.376660,4297,852.711,...,195.393228,686.603,159.823790,233.595021,482,3.783077,6.528288,3.468227,3.662444,3.565336
4,p12,2,1,0.695652,11,3.736884,3.793618,3.765251,2333,453.578,...,179.762517,520.548,223.219554,318.248713,482,3.783077,6.528288,3.800883,3.832909,3.816896
5,p35,1,1,0.608696,9,3.805560,3.937057,3.871308,3763,800.892,...,140.916436,1476.111,392.374003,1488.871060,482,3.783077,6.528288,3.827092,3.966741,3.896917
6,p10,1,2,0.608696,17,3.952658,4.157124,4.054891,4928,1097.945,...,191.733506,640.565,130.011163,171.306512,482,3.783077,6.528288,3.959827,4.399944,4.179886
7,p17,2,2,0.608696,22,3.823212,3.335250,3.579231,1347,357.818,...,174.076721,473.116,351.497771,857.220751,482,3.783077,6.528288,4.144958,3.613001,3.878979
8,p1,1,1,0.608696,26,3.607191,3.515121,3.561156,2422,357.453,...,188.542741,1927.822,796.291615,2085.008822,482,3.783077,6.528288,3.508404,3.418535,3.463469
9,p34,1,2,0.608696,38,3.401875,3.562814,3.482345,6902,1502.960,...,210.692690,1419.637,205.714679,1153.503760,482,3.783077,6.528288,3.585290,3.817738,3.701514


### Bottom 10

In [33]:
bottom10 = add_data.copy()
bottom10.sort_values(by=['Task_Success','Time_On_Task'], ascending=[True,False], inplace=True)
bottom10.reset_index(drop=True, inplace=True)
bottom10 = bottom10.head(10)
bottom10

,ID,Ontologies,Visualization,Task_Success,Time_On_Task
0,p27,2,2,0.086957,50
1,p7,2,2,0.086957,20
2,p14,2,1,0.217391,28
3,p27,1,1,0.260870,13
4,p32,1,2,0.304348,21
5,p12,1,2,0.304348,17
6,p25,2,2,0.347826,39
7,p11,2,2,0.347826,33
8,p33,2,2,0.347826,27
9,p2,1,2,0.347826,19


In [34]:
bottom10.groupby(by=['Ontologies','Visualization']).mean()

Task_Success  Time_On_Task
Ontologies Visualization                            
1          1                  0.260870          13.0
           2                  0.318841          19.0
2          1                  0.217391          28.0
           2                  0.243478          33.8

In [35]:
%%time
bottom10.apply(get_metrics_df, axis=1)

27
7
14
27
32
12
25
11
33
2
Wall time: 39.9 s


,ID,Ontologies,Visualization,Task_Success,Time_On_Task,base_avg_size_left,base_avg_size_right,base_avg_size_both,total_fixations,sum_fixation_duration,...,std_saccade_length,sum_saccade_duration,mean_saccade_duration,std_saccade_duration,total_left_clicks,mean_time_between_clicks,std_time_between_clicks,avg_size_left,avg_size_right,avg_size_both
0,p27,2,2,0.086957,50,3.963408,3.888980,3.926194,1614,567.699,...,227.450055,202.986,125.843769,718.178810,482,3.783077,6.528288,4.397918,4.394627,4.396272
1,p7,2,2,0.086957,20,3.688890,3.613177,3.651034,2711,673.240,...,187.148021,333.584,123.093727,259.030213,482,3.783077,6.528288,3.694683,3.515025,3.604854
2,p14,2,1,0.217391,28,3.429302,3.492846,3.461074,2234,533.086,...,157.064601,391.157,175.171070,237.558901,482,3.783077,6.528288,3.749434,3.564784,3.657109
3,p27,1,1,0.260870,13,3.963408,3.888980,3.926194,3568,899.321,...,187.939688,2011.514,563.923185,13649.262083,482,3.783077,6.528288,4.126828,4.097416,4.112122
4,p32,1,2,0.304348,21,4.015239,3.822404,3.918821,1735,597.500,...,171.671392,97.481,56.217416,98.664361,482,3.783077,6.528288,3.742211,3.637580,3.689895
5,p12,1,2,0.304348,17,3.736884,3.793618,3.765251,1132,205.727,...,183.145269,459.508,406.284704,693.935532,482,3.783077,6.528288,3.810049,3.866253,3.838151
6,p25,2,2,0.347826,39,4.033584,4.339193,4.186388,4788,1565.245,...,236.612130,459.428,95.974097,147.312425,482,3.783077,6.528288,4.029822,4.414277,4.222049
7,p11,2,2,0.347826,33,4.106171,3.731255,3.918713,1152,165.166,...,230.676812,771.717,670.475239,1858.738311,482,3.783077,6.528288,3.908292,3.531979,3.720135
8,p33,2,2,0.347826,27,3.147223,2.981043,3.064133,2849,570.506,...,192.879931,512.200,179.845506,252.721830,482,3.783077,6.528288,3.189525,3.047665,3.118595
9,p2,1,2,0.347826,19,3.073719,2.774450,2.924085,2581,414.829,...,190.863949,1149.467,445.529845,570.704203,482,3.783077,6.528288,3.057603,2.781126,2.919364
